# Exploratory Data Analysis

In [1]:

import os
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def fileCounter(directory: str):
    lst = os.listdir(directory)
    fileCount = len(lst)
    
    return fileCount

# First attepmt to train
- Lasso Regularization
- CNN Network
- 3 x 256 x 256 input size

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.transforms.functional as F
from torchvision.transforms import v2

import torchmetrics as metrics

from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image